# 设计一个具有2个卷积层的卷积神经网络

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F

# 定义卷积神经网络
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)  # 第一个卷积层
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  # 第二个卷积层
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)  # 最大池化层
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # 第一个全连接层
        self.fc2 = nn.Linear(128, 10)  # 第二个全连接层

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 卷积 -> ReLU -> 池化
        x = self.pool(F.relu(self.conv2(x)))  # 卷积 -> ReLU -> 池化
        x = x.view(-1, 64 * 7 * 7)  # 展平
        x = F.relu(self.fc1(x))  # 全连接层 -> ReLU
        x = self.fc2(x)  # 全连接层
        return x

# 实例化模型
model = CNN()
print(model)

# 构造卷积神经网络的前向传播和后向传播程序

In [ ]:
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# 定义前向传播和后向传播过程
def train(model, trainloader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()  # 清除梯度
            outputs = model(inputs)  # 前向传播
            loss = criterion(outputs, labels)  # 计算损失
            loss.backward()  # 后向传播
            optimizer.step()  # 更新参数

            running_loss += loss.item()
            if i % 200 == 199:  # 每200个batch打印一次损失
                print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 200:.3f}")
                running_loss = 0.0
    print('Finished Training')

# 训练卷积神经网络

In [ ]:
# 加载 MNIST 数据集
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# 训练模型
train(model, trainloader, criterion, optimizer, epochs=5)

# 利用卷积神经网络实现 MNIST 手写体识别，并与多层感知器结果进行比较

In [ ]:
import numpy as np

# 测试卷积神经网络
def test(model, testloader):
    model.eval()  # 设置为评估模式
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

# 测试模型
test(model, testloader)

# 可视化一些测试结果
dataiter = iter(testloader)
images, labels = next(dataiter)  # 使用 next 函数
outputs = model(images)
_, predicted = torch.max(outputs, 1)

# 显示图片
def imshow(img):
    img = img / 2 + 0.5  # 反归一化
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# 打印前5个图像及其预测标签
imshow(torchvision.utils.make_grid(images[:5]))
print('GroundTruth: ', ' '.join(f'{labels[j]}' for j in range(5)))
print('Predicted: ', ' '.join(f'{predicted[j]}' for j in range(5)))